In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://github.com/karoldvl/ESC-50/archive/master.zip
!unzip master.zip

--2025-06-06 12:08:49--  https://github.com/karoldvl/ESC-50/archive/master.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/karolpiczak/ESC-50/archive/master.zip [following]
--2025-06-06 12:08:49--  https://github.com/karolpiczak/ESC-50/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master [following]
--2025-06-06 12:08:50--  https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.116.10
Connecting to codeload.github.com (codeload.github.com)|140.82.116.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 645695005 (616M) [application/zip]
Saving to: ‘master.zip’

master.zip          100

In [ ]:
!pip install librosa soundfile

In [ ]:
import librosa
import soundfile as sf
import os
import random
import numpy as np

In [ ]:
def mix_with_noise(voice_path, noise_path, output_path, snr_db=10):
    voice, sr = librosa.load(voice_path, sr=None)
    noise, _ = librosa.load(noise_path, sr=sr)

    if len(noise) < len(voice):
        repeat_count = int(np.ceil(len(voice) / len(noise)))
        noise = np.tile(noise, repeat_count)
    noise = noise[:len(voice)]

    voice_power = np.mean(voice**2)
    noise_power = np.mean(noise**2)

    if noise_power < 1e-10:
        print(f"⚠️ Skipped noise file with near-zero energy: {noise_path}")
        return False

    snr_linear = 10 ** (snr_db / 10)
    scale = np.sqrt(voice_power / (snr_linear * noise_power))
    noise = noise * scale

    noisy_voice = voice + noise

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    sf.write(output_path, noisy_voice, sr)
    return True


In [ ]:
# Sample CREMA-D voice file
voice_file = '/content/drive/MyDrive/FYP/Datasets/Crema D/AudioWAV/1001_DFA_ANG_XX.wav'  # Replace with your path

# Random noise file from ESC-50
noise_files = os.listdir('/content/ESC-50-master/audio/')
random_noise_file = random.choice(noise_files)
noise_file = os.path.join('/content/ESC-50-master/audio/', random_noise_file)

# Output path
output_file = '/content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/1001_IEO_HAP_noisy.wav'

# Mix and save
mix_with_noise(voice_file, noise_file, output_file)
print("✅ Noisy file created:", output_file)


✅ Noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/1001_IEO_HAP_noisy.wav


In [ ]:
from IPython.display import Audio
Audio(output_file)

In [ ]:
input_dir = '/content/drive/MyDrive/FYP/Datasets/Crema D/AudioWAV'
noise_dir = '/content/ESC-50-master/audio'

output_dir ={
    10: '/content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-10db',
    5: '/content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-5db',
    0: '/content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-0db'
}

for filename in os.listdir(input_dir):
    if filename.endswith('.wav'):
        voice_path = os.path.join(input_dir, filename)

        for snr in [10, 5, 0]:
            noise_file = random.choice(os.listdir(noise_dir))
            noise_path = os.path.join(noise_dir, noise_file)
            output_path = os.path.join(output_dir[snr], filename.replace('.wav', f'_{snr}db.wav'))

            success = mix_with_noise(voice_path, noise_path, output_path, snr_db=snr)
            if success:
                print(f"✅ {snr}dB noisy file created: {output_path}")
            else:
                print(f"❌ Skipped: {filename} with {snr}dB")


Streaming output truncated to the last 5000 lines.
✅ 5dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-5db/1059_IEO_HAP_LO_5db.wav
✅ 0dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-0db/1059_IEO_HAP_LO_0db.wav
✅ 10dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-10db/1059_IEO_FEA_MD_10db.wav
✅ 5dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-5db/1059_IEO_FEA_MD_5db.wav
✅ 0dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-0db/1059_IEO_FEA_MD_0db.wav
✅ 10dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-10db/1059_IOM_FEA_XX_10db.wav
✅ 5dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-5db/1059_IOM_FEA_XX_5db.wav
✅ 0dB noisy file created: /content/drive/MyDrive/FYP/Datasets/CREMA-D-noisy/CREMA-D-noisy-0db/1059_IOM_FEA_XX_0db.wav

In [13]:
for snr in [10, 5, 0]:
    folder = output_dir[snr]
    count = len([f for f in os.listdir(folder) if f.endswith('.wav')])
    print(f"🎧 Files at {snr}dB: {count}")

🎧 Files at 10dB: 7439
🎧 Files at 5dB: 7439
🎧 Files at 0dB: 7438
